In [2]:


# 🧰 Install Dependencies
!pip install -q keras-tuner

# 📦 Imports
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers, callbacks, initializers
import keras_tuner as kt
import matplotlib.pyplot as plt
import numpy as np

# 🧹 Set Random Seeds
np.random.seed(42)
tf.random.set_seed(42)

# 📥 Load & Prepare Fashion MNIST
def load_data():
    (x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()
    x_train = x_train.astype("float32") / 255.0
    x_test = x_test.astype("float32") / 255.0
    x_train = np.expand_dims(x_train, -1)
    x_test = np.expand_dims(x_test, -1)
    x_val = x_train[-10000:]; y_val = y_train[-10000:]
    x_train = x_train[:-10000]; y_train = y_train[:-10000]
    return x_train, y_train, x_val, y_val, x_test, y_test

x_train, y_train, x_val, y_val, x_test, y_test = load_data()

# 🧪 Utility: Plot History
def plot_history(history, title):
    plt.plot(history.history['val_accuracy'], label='val acc')
    plt.plot(history.history['accuracy'], label='train acc')
    plt.title(title)
    plt.legend()
    plt.show()

# 🔧 A/B Test Helper
def create_model(regularizer=None, dropout_rate=0.0, initializer='glorot_uniform', batch_norm=False):
    model = keras.Sequential()
    model.add(layers.Input(shape=(28,28,1)))
    model.add(layers.Conv2D(32, 3, activation='relu', kernel_regularizer=regularizer, kernel_initializer=initializer))
    if batch_norm:
        model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D())
    model.add(layers.Flatten())
    model.add(layers.Dropout(dropout_rate))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# 🔁 Early Stopping Callback
early_stop = callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# 🧪 Experiment 1: L2 Regularization
model_l2 = create_model(regularizer=regularizers.l2(0.01))
history_l2 = model_l2.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=10, callbacks=[early_stop])
plot_history(history_l2, "L2 Regularization")

# 🧪 Experiment 2: Dropout
model_dropout = create_model(dropout_rate=0.5)
history_dropout = model_dropout.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=10, callbacks=[early_stop])
plot_history(history_dropout, "Dropout")

# 🧪 Experiment 3: Batch Normalization
model_bn = create_model(batch_norm=True)
history_bn = model_bn.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=10, callbacks=[early_stop])
plot_history(history_bn, "Batch Normalization")

# 🧪 Experiment 4: Initializers
for init in ['he_uniform', 'lecun_normal']:
    model_init = create_model(initializer=init)
    history_init = model_init.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=10, callbacks=[early_stop])
    plot_history(history_init, f"Initializer: {init}")

# 🧪 Experiment 5: Monte Carlo Dropout (Custom Layer)
class MCDropout(layers.Dropout):
    def call(self, inputs, training=None):
        return super().call(inputs, training=True)

mc_model = keras.Sequential([
    layers.Input(shape=(28,28,1)),
    layers.Conv2D(32, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    MCDropout(0.5),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])
mc_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history_mc = mc_model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=10, callbacks=[early_stop])
plot_history(history_mc, "Monte Carlo Dropout")

# 🧪 Custom Regularizer Example
class CustomL1(tf.keras.regularizers.Regularizer):
    def __init__(self, factor):
        self.factor = factor
    def __call__(self, x):
        return self.factor * tf.reduce_sum(tf.abs(x))

model_custom_reg = create_model(regularizer=CustomL1(0.001))
history_custom_reg = model_custom_reg.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=10, callbacks=[early_stop])
plot_history(history_custom_reg, "Custom L1 Regularizer")

# 🧪 TensorBoard + Callback Demo
logdir = "logs"
tb_callback = callbacks.TensorBoard(log_dir=logdir)
model_tb = create_model()
model_tb.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=5, callbacks=[tb_callback])

# 🧪 Keras Tuner Demo
def model_builder(hp):
    hp_units = hp.Int("units", min_value=32, max_value=256, step=32)
    model = keras.Sequential([
        layers.Flatten(input_shape=(28, 28, 1)),
        layers.Dense(hp_units, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])
    hp_lr = hp.Choice("learning_rate", [1e-2, 1e-3, 1e-4])
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_lr),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

tuner = kt.Hyperband(model_builder, objective='val_accuracy', max_epochs=5, directory='my_dir', project_name='keras_tune')
tuner.search(x_train, y_train, epochs=5, validation_data=(x_val, y_val))

best_model = tuner.get_best_models(1)[0]
loss, acc = best_model.evaluate(x_test, y_test)
print(f"Best tuned model test accuracy: {acc:.4f}")


Trial 10 Complete [00h 00m 26s]
val_accuracy: 0.8562999963760376

Best val_accuracy So Far: 0.8805000185966492
Total elapsed time: 00h 02m 29s
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8698 - loss: 0.3589
Best tuned model test accuracy: 0.8694
